In [1]:
import pandas as pd
import numpy as np
import pygad
import random
import copy

In [2]:
import csv

# Initialize an empty dictionary
enginge_RUL = {}

with open('RUL_consultancy_predictions_A3-2.csv', newline='') as csvfile:
    # Create a CSV reader object with semicolon as delimiter
    reader = csv.reader(csvfile, delimiter=';')
    # Skip the header row
    next(reader)
    # Loop over each row in the CSV
    for row in reader:
        # Check if row has exactly 2 columns
        if len(row) == 2:
            enginge_RUL[int(row[1])] = int(row[0])

In [3]:
engine_RUL_30 = {key: value for key, value in enginge_RUL.items() if value <= 30}

In [4]:
Team_A_time = {key: 4 for key in range(1, 21)}
Team_A_time.update({key: 3 for key in range(21, 56)})
Team_A_time.update({key: 2 for key in range(56, 81)})
Team_A_time.update({key: 8 for key in range(81, 101)})

Team_B_time = {j : Team_A_time[j] + 1 for j in range(1, 26)}
Team_B_time.update({j : Team_A_time[j] + 2 for j in range(26, 71)})
Team_B_time.update({j : Team_A_time[j] + 1 for j in range(71, 101)})

In [5]:
T1_time = Team_A_time
T2_time = Team_B_time
T3_time = Team_A_time
T4_time = Team_B_time

# Random Initializer

In [6]:
def calculate_team_work_time(team, engines):
    if team in ["T1", "T3"]:
        engine_times = Team_A_time
    else:
        engine_times = Team_B_time
    
    time = 0
    
    for engine in engines:
        time += engine_times[engine]
    
    return time
    

def random_init(engines_RUL):
    schedule = {"T1":[], "T2":[], "T3":[], "T4":[]}
    
    possible_engines = list(engines_RUL.keys())
    possible_teams = ["T1", "T2", "T3", "T4"]
    
    iterations = 0 
    
    while possible_engines and iterations < 500:
        index = random.randrange(4)
        team = possible_teams[index]
        
        index = random.randrange(len(possible_engines))
        selected_engine = possible_engines[index]
        
        schedule[team].append(selected_engine)
        
        possible_engines.remove(selected_engine)
        
        iterations += 1

    return schedule

In [7]:
random_init(engine_RUL_30)

{'T1': [64, 76, 66, 68, 90, 34, 49],
 'T2': [56, 41, 35, 82, 81],
 'T3': [36, 61, 77, 20, 91, 37, 92],
 'T4': [31, 24, 100, 42, 40]}

# Loss Function

In [8]:
def engine_fix_date(schedule):
    engine_fix = {}
    for team in schedule:
        days = 0
        if team == "T1" or team == "T3":
            time_table = Team_A_time
        else:
            time_table = Team_B_time
            
        for engine in schedule[team]:
            engine_fix[engine] = days
            days = days + time_table[engine]
              
    return engine_fix


initial_schedule = random_init(engine_RUL_30)
print(initial_schedule)

engine_cost = {key: 4 for key in range(1, 21)}
engine_cost.update({key: 3 for key in range(21, 31)})
engine_cost.update({key: 2 for key in range(31, 46)})
engine_cost.update({key: 5 for key in range(46, 81)})
engine_cost.update({key: 6 for key in range(81, 100)})


def loss_function(schedule, engine_RUL, engine_cost):
    loss = 0
    fix_dates = engine_fix_date(schedule)
    print(fix_dates)
    print(engine_RUL)
    for team in schedule:
        for engine in schedule[team]:

            days_late = fix_dates[engine] - engine_RUL[engine]
            engine_loss = 0
            if days_late > 0:
                cost = engine_cost[engine]
    
                for i in range(days_late):
                    day_loss = cost * (i ** 2)
                    if day_loss > 250:
                        day_loss = 250
                    engine_loss += day_loss
            loss += engine_loss
        
    return loss  

loss_schedule = loss_function(initial_schedule, engine_RUL_30, engine_cost)
print(loss_schedule)

{'T1': [36, 41, 82, 35, 77, 64, 61, 90, 34], 'T2': [92, 68, 20, 100, 91, 31, 81], 'T3': [56, 76, 49, 42, 66, 24], 'T4': [40, 37]}
{36: 0, 41: 3, 82: 6, 35: 14, 77: 17, 64: 19, 61: 21, 90: 23, 34: 31, 92: 0, 68: 9, 20: 13, 100: 18, 91: 27, 31: 36, 81: 41, 56: 0, 76: 2, 49: 4, 42: 7, 66: 10, 24: 12, 40: 0, 37: 5}
{20: 10, 24: 26, 31: 14, 34: 8, 35: 8, 36: 24, 37: 21, 40: 29, 41: 23, 42: 13, 49: 14, 56: 18, 61: 23, 64: 24, 66: 18, 68: 13, 76: 3, 77: 27, 81: 6, 82: 11, 90: 28, 91: 29, 92: 24, 100: 24}
14950


# Termination Criterion

In [9]:
# TO DO LATER

# Selection of parents

In [10]:
# TO DO SELECT ONE DROM PYGAD UTILS
# https://pygad.readthedocs.io/en/latest/pygad.html#parent-selection-methods

# Crossover

In [76]:
# maybe TO DO: ONLY DELETE THE ORIGINAL DUPLICATES, NOT THE SWAPPED DUPLICATES

def double_fixer(child, parent):
    doubles = []
    skipped_ones = []
    numbers = {}
    
    for team in parent:
        for engine in parent[team]:
            numbers[engine] = [0]
    
    for team in child:
        index = -1
        for engine in child[team]:
            index = index + 1
            numbers[engine][0] = numbers[engine][0] + 1
            numbers[engine].append(team)
            numbers[engine].append(index)
                
    for key in numbers:
        if numbers[key][0] == 0:
            skipped_ones.append(key)
        elif numbers[key][0] > 1:
            doubles.append(key)
    
    for fault in doubles:
        team = numbers[fault][-2]
        index = numbers[fault][-1]        
        next_missed_one = skipped_ones.pop()
        child[team][index] = next_missed_one
        
    return child


def find_center(parent, team, middle_down_round):
    if len(parent[team]) % 2 == 0:
            middle_index = len(parent[team]) // 2
            next_middle_down_round = middle_down_round
    else:
        if middle_down_round:
            middle_index = len(parent[team]) // 2             
            next_middle_down_round = False
        else: 
            middle_index = (len(parent[team]) // 2) + 1
            next_middle_down_round = True
    
    return middle_index, next_middle_down_round


def crossover(parents, offspring_size = 2):
    parent_1 = parents[0]
    parent_2 = parents[1]
    child1 = {key: [] for key in parent_1}
    child2 = {key: [] for key in parent_2} 
    
    cross_side = random.choice(["Left", "Right"]) # Right means you keep the right side of p1 and switch the right side of p2
    print(cross_side)
    
    p1_middle_down_round = True
    p2_middle_down_round = True
    
    for team in parent_1:
        
        p1_middle, p1_middle_down_round = find_center(parent_1, team, p1_middle_down_round)
        p2_middle, p2_middle_down_round = find_center(parent_2, team, p2_middle_down_round)
        
        if cross_side == "Left":
            sub_schedule_p1 = parent_1[team][:p1_middle]
            sub_schedule_p2 = parent_2[team][:p2_middle]
            
            child1[team].extend(sub_schedule_p2)
            child1[team].extend(parent_1[team][p1_middle:])
            
            child2[team].extend(sub_schedule_p1)
            child2[team].extend(parent_2[team][p2_middle:])
        
        if cross_side == "Right":
            sub_schedule_p1 = parent_1[team][p1_middle:]
            sub_schedule_p2 = parent_2[team][p2_middle:]
            
            child1[team].extend(parent_1[team][:p1_middle])
            child1[team].extend(sub_schedule_p2)
            
            child2[team].extend(parent_2[team][:p2_middle])
            child2[team].extend(sub_schedule_p1)
    
    child1 = double_fixer(child1, parent_1)
    child2 = double_fixer(child2, parent_2)
    
    print(sum(len(value) for value in child1.values()))
    print(sum(len(value) for value in child2.values()))
    
    return child1, child2    

In [101]:
parent_1 = random_init(engine_RUL_30)
parent_2 = random_init(engine_RUL_30)

#print_parent_formatted(parent_1, "parent_1")
#print_parent_formatted(parent_2, "parent_2")

crossover(parents = [parent_1, parent_2])

Left
24
24


({'T1': [82, 61, 77, 36],
  'T2': [90, 42, 35, 66, 31, 64],
  'T3': [81, 37, 56, 76, 41, 34, 91, 49],
  'T4': [40, 20, 100, 24, 92, 68]},
 {'T1': [61, 81, 100, 66, 56],
  'T2': [40, 76, 64, 24],
  'T3': [90, 91, 42, 34, 20, 92, 77, 37, 68, 35],
  'T4': [49, 31, 41, 36, 82]})

In [98]:
def print_parent_formatted(parent, name):
    formatted = ',\n  '.join(f"'{key}': {value}" for key, value in parent.items())
    print(f"{name} = {{\n  {formatted}\n}}")

In [68]:
print(sum(len(value) for value in child1.values()))
print(sum(len(value) for value in child2.values()))

24
24


In [15]:
test = {}
test.update({1 : 0})
test.update({2 : 0})
print(test[1]) 
test[1] = 4
test[1] = test[1] + 1
test[5]
print(test)

0


KeyError: 5

In [ ]:
parent_1 = random_init(engine_RUL_30)

child1 = {key: [] for key in parent_1}
child1['T1'].extend([1,2,3])
child1['T1'].extend([4,5,6])
child1

{'T1': [1, 2, 3, 4, 5, 6], 'T2': [], 'T3': [], 'T4': []}